In [221]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import pandas as pd
import time
import re
import urllib3
import concurrent.futures
import multiprocessing

In [222]:
# input df
data = pd.read_csv("wsi_map.csv")

In [223]:
# set variables
MAX_THREADS = multiprocessing.cpu_count()
BUCKET_SIZE = len(data["news_link"]) / (MAX_THREADS * 8)
OUTPUT_FILE = "result.csv"

In [224]:
def get_row(link):
    """
        Function to scrape WSI that takes in input a link a WSI news link and return a dict with getting section, tag,                              category, description
    """
    response = http.request("GET",link)
    soup = BeautifulSoup(response.data, 'html.parser')

    row_dict = { "link": link,
        "section": [ item.get("content", None) for item in soup.findAll("meta", { "property" : re.compile(r"section")}) ],
        "tag": [ item.get("content", None) for item in soup.findAll("meta", { "property" : re.compile(r"tag")}) ],
        "category": [item.text for item in soup.find("div", {"class":"category"}).findAll("li") ],
        "description": soup.find("meta", {"name": re.compile(r"description")}).get("content", None)
        }
    
    return row_dict
    


def exec_threads(function, serie, file_name):
    """
        Function that divide the execution in threads and save the output that takes 3 input: a scrape function, a pd links serie, and a            csv filename
    """
    threads = min(MAX_THREADS, len(serie))
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        result = pd.DataFrame(executor.map(function, serie)).to_csv(file_name, mode='a', header=False)



def main(urls):

    t0 = time.time()
    exec_threads(get_row,urls,OUTPUT_FILE)
    t1 = time.time()
    print(f"---- {t1-t0} seconds ---- ")

In [ ]:
print(f"executing in {MAX_THREADS} threads ")
### divide the execution in bucket
for serie in np.split(data["news_link"], BUCKET_SIZE):
    main(serie)